In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np


In [22]:
diabetes = pd.read_csv("data/diabetes_prediction_dataset.csv")

diabetes = diabetes[["gender", "Age", "BMI", "blood_glucose_levels", "diabetes"]]

diabetes

,gender,Age,BMI,blood_glucose_levels,diabetes
0,Female,67.0,27.32,80,0.0
1,Female,58.0,27.32,90,0.0
2,Male,2.0,18.41,100,0.0
3,Male,26.0,27.32,100,0.0
4,Female,61.0,24.07,160,0.0
...,...,...,...,...,...
49995,Female,80.0,27.32,90,0.0
49996,Female,2.0,17.37,100,0.0
49997,Male,66.0,27.83,155,0.0
49998,Female,24.0,35.42,100,0.0


In [23]:
diabetes.to_csv("data/diabetes_tidied.csv")

In [24]:
diabetes_tidied = pd.read_csv("data/diabetes_tidied.csv")

In [25]:
diabetes_train, diabetes_test = train_test_split(diabetes_tidied, train_size=.75)

In [26]:
diabetes_train.to_csv("data/diabetes_tidied_train.csv")

In [27]:
diabetes_train = diabetes_train[["gender", "Age", "BMI", "blood_glucose_levels", "diabetes"]]

In [28]:
diabetes_test = diabetes_test[["gender", "Age", "BMI", "blood_glucose_levels", "diabetes"]]

In [29]:
diabetes_train.to_csv("data/diabetes_tidied_train.csv")

In [30]:
diabetes_test.to_csv("data/diabetes_tidied_test.csv")

In [31]:
diabetes_train = diabetes_train.reset_index(drop=True)

In [32]:
diabetes_train

,gender,Age,BMI,blood_glucose_levels,diabetes
0,Female,64.00,30.04,90,0.0
1,Male,76.00,28.23,126,0.0
2,Female,27.00,30.66,126,0.0
3,Male,33.00,25.34,90,0.0
4,Male,1.72,16.57,130,0.0
...,...,...,...,...,...
37495,Female,75.00,32.53,200,0.0
37496,Female,80.00,27.32,200,0.0
37497,Female,12.00,26.93,130,0.0
37498,Male,52.00,19.58,100,0.0


**This marks the beginning of Visualization and table analysis**

In [33]:
grouped = diabetes_train.groupby(['gender', 'diabetes']).size().reset_index(name='Count')

In [34]:
grouped

,gender,diabetes,Count
0,Female,0.0,20254
1,Female,1.0,1631
2,Male,0.0,14105
3,Male,1.0,1505
4,Other,0.0,5


In [35]:
diabetes_train

,gender,Age,BMI,blood_glucose_levels,diabetes
0,Female,64.00,30.04,90,0.0
1,Male,76.00,28.23,126,0.0
2,Female,27.00,30.66,126,0.0
3,Male,33.00,25.34,90,0.0
4,Male,1.72,16.57,130,0.0
...,...,...,...,...,...
37495,Female,75.00,32.53,200,0.0
37496,Female,80.00,27.32,200,0.0
37497,Female,12.00,26.93,130,0.0
37498,Male,52.00,19.58,100,0.0


In [36]:
diabetes_female = diabetes_train[diabetes_train['gender'] == "Female"]
diabetes_male = diabetes_train[diabetes_train['gender'] == "Male"]

In [37]:
avg_bmi_FWdiabetes = diabetes_female[diabetes_female['diabetes'] == 1.0]["BMI"].mean()
avg_bmi_FWOUTdiabetes = diabetes_female[diabetes_female['diabetes'] == 0.0]["BMI"].mean()
avg_BGL_FWdiabetes= diabetes_female[diabetes_female['diabetes'] == 1.0]["blood_glucose_levels"].mean()
avg_BGL_FWOUTdiabetes= diabetes_female[diabetes_female['diabetes'] == 0.0]["blood_glucose_levels"].mean()

avg_bmi_MWdiabetes = diabetes_male[diabetes_male['diabetes'] == 1.0]["BMI"].mean()
avg_bmi_MWOUTdiabetes = diabetes_male[diabetes_male['diabetes'] == 0.0]["BMI"].mean()
avg_BGL_MWdiabetes = diabetes_male[diabetes_male['diabetes'] == 1.0]["blood_glucose_levels"].mean()
avg_BGL_MWOUTdiabetes = diabetes_male[diabetes_male['diabetes'] == 0.0]["blood_glucose_levels"].mean()

avg_bmi_MWdiabetes = diabetes_male[diabetes_male['diabetes'] == 1.0]["Age"].mean()
avg_bmi_MWOUTdiabetes = diabetes_male[diabetes_male['diabetes'] == 0.0]["Age"].mean()

avg_AGE_FWdiabetes= diabetes_female[diabetes_female['diabetes'] == 1.0]["Age"].mean()
avg_AGE_FWOUTdiabetes= diabetes_female[diabetes_female['diabetes'] == 0.0]["Age"].mean()

In [38]:
grouped = grouped.assign(BMI_averages = [avg_bmi_FWOUTdiabetes, avg_bmi_FWdiabetes, avg_bmi_MWOUTdiabetes, avg_bmi_MWdiabetes, 0])
grouped = grouped.assign(BGL_averages = [avg_BGL_FWOUTdiabetes, avg_BGL_FWdiabetes, avg_BGL_MWOUTdiabetes, avg_BGL_MWdiabetes, 0])
grouped = grouped.assign(AGE_averages = [avg_AGE_FWOUTdiabetes, avg_AGE_FWdiabetes, avg_bmi_MWOUTdiabetes, avg_bmi_MWdiabetes, 0])

grouped

,gender,diabetes,Count,BMI_averages,BGL_averages,AGE_averages
0,Female,0.0,20254,27.051835,133.059840,40.941610
1,Female,1.0,1631,32.391196,195.107296,61.581239
2,Male,0.0,14105,39.114016,133.273449,39.114016
3,Male,1.0,1505,60.636545,194.227907,60.636545
4,Other,0.0,5,0.000000,0.000000,0.000000


**EVERYTHING BELOW IS PART OF VISUALIZING OUR DATA**

In [39]:
import altair as alt
import pandas as pd
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [40]:
diabetes = pd.read_csv("data/diabetes_tidied_train.csv")
diabetes = diabetes[diabetes["gender"] != "Other"]
diabetes

,Unnamed: 0,gender,Age,BMI,blood_glucose_levels,diabetes
0,48590,Female,64.00,30.04,90,0.0
1,13833,Male,76.00,28.23,126,0.0
2,28828,Female,27.00,30.66,126,0.0
3,3090,Male,33.00,25.34,90,0.0
4,30896,Male,1.72,16.57,130,0.0
...,...,...,...,...,...,...
37495,11801,Female,75.00,32.53,200,0.0
37496,27835,Female,80.00,27.32,200,0.0
37497,20861,Female,12.00,26.93,130,0.0
37498,36042,Male,52.00,19.58,100,0.0


In [41]:
chart_bmi = alt.Chart(diabetes, title="Distribution of BMI").mark_bar().encode(
    x = alt.X("BMI", bin=alt.Bin(maxbins = 20), scale=alt.Scale(domain=[10, 80])),
    y = alt.Y("count()", title = "Count")#.title("Count")
) 

chart_bmi

In [ ]:
chart_age = alt.Chart(diabetes, title="Distribution of Age").mark_bar().encode(
    x = alt.X("Age", bin=alt.Bin(maxbins = 30), title = "Age (years, binned)"),
    y = alt.Y("count()", title = "Count")
)
chart_age

In [ ]:
chart_glucose = alt.Chart(diabetes, title="Distribution of Blood Glucose Levels").mark_bar().encode(
    x = alt.X("blood_glucose_levels", bin=alt.Bin(maxbins = 20), title = "Blood Glucose Level (random testing, mg/dl)"),
              #, scale=alt.Scale(domain=[10, 60])
             #).title("Blood Glucose Level (random testing, mg/dl)"),
    y = alt.Y("count()", title = "Count")
)
chart_glucose

In [ ]:
chart_gender = alt.Chart(diabetes, title="Gender").mark_bar().encode(
    x = alt.X("gender"
              #, scale=alt.Scale(domain=[10, 60])
             , title = "Gender"),
    y = alt.Y("count()", title = "Count")
)
chart_gender

In [ ]:
chart_diabetes = alt.Chart(diabetes, title="Diagnosis").mark_bar().encode(
    x = alt.X("diabetes", bin=alt.Bin(maxbins = 2)
              , scale=alt.Scale(domain=[0, 1]), title = "Diabetes (0 = no, 1 = yes)"
             ),
    y = alt.Y("count()", title = "Count")
)
chart_diabetes

In [ ]:
chart_bmi | chart_age | chart_gender | chart_glucose | chart_diabetes

*The table below named GROUPED is the final table has summarized significant values within the dataset.*

*The Visualization below is the final summary of all valid visuals to analyze our data.*

In [ ]:
grouped

NameError: name 'grouped' is not defined